In [1]:
import numpy as np
import pandas as pd
import random as r
import math
import os
from datetime import timedelta
import statistics as stats 

import matplotlib.pyplot as plot
import matplotlib.ticker as mtick
import networkx as nx
import seaborn as sns

from pyomo.environ import *
from pyomo.opt import SolverFactory
#from gurobipy import GRB

In [2]:
os.chdir(os.getcwd()+'/outputs/strategic_decisions')
#os.chdir(os.getcwd()+'/outputs/ordering_plans')

In [3]:
budget_levels = list(range(1000000, 3000000+1, 500000))

reliability_levels = [.5,.75,.95]

warehouse_cap_level = 1 #time intervals average amount of supply that can be stored

trade_off_dict = {}

I_set = 4
K_set = 1
T_set = 12

I_K_dict = {1: [1,2,3,4]}

In [4]:
def initialize_model(reliability_level, 
                     budget_level):
    ###health care commodity metric parameters####
    #criticality score
    
    I_set = 4
    K_set = 1
    T_set = 12

    I_K_dict = {1: [1,2,3,4]}

    model = ConcreteModel()

    #####define sets#######
    model.K = Set(initialize = range(1,K_set+1))
    model.I = Set(initialize = range(1,I_set+1))
    model.T = Set(initialize = range(1,T_set+1))
    model.T_0 = Set(initialize = range(0,T_set+1))
    model.I_K = Set(model.K)
    for k in model.K:
        for i in range(0,len(I_K_dict[k])):
            model.I_K[k].add(I_K_dict[k][i])
    
    
    r_k_dict = {
        1 : .5
    }

    #suitability score (only specified for suitable items)
    q_k_i_dict = {
        tuple([1,1]):1,
        tuple([1,2]):.9,
        tuple([1,3]):.8,
        tuple([1,4]):.1
    }

    def rank_score_initialize(model, k):
        return(r_k_dict.get(k))

    model.r_k = Param(model.K, initialize = rank_score_initialize)

    def criticality_discounting_metric_initialize(model, k, t):
        tau_k_t_temp = (model.r_k[k])*((1+model.r_k[k])**(T_set-t))

        return(tau_k_t_temp)

    model.tau_k_t = Param(model.K, model.T, initialize = criticality_discounting_metric_initialize)

    def suitability_score_initialize(model, k, i):
        if (i in model.I_K[k]):
            return(q_k_i_dict.get(tuple([k,i])))
        else:
            return(None)

    model.q_k_i = Param(model.K, model.I, initialize = suitability_score_initialize)

    #######warehouse capacity parameters#######

    #size
    s_i_dict = {
        1: 1,
        2: 1,
        3: 1,
        4: 1
    }

    #total warehouse capacity
    h = 50000

    def item_size_param_initialize(model, i):
        return(s_i_dict.get(i))

    model.s_i = Param(model.I, initialize = item_size_param_initialize)

    #model.s_k.pprint()

    def warehouse_cap_param_initialize(model):
        return(h)

    model.h = Param(initialize = h)

    #######budget parameters########

    #cost
    c_i_dict = {
        1: 8,
        2: 5,
        3: 10,
        4: 1
    }

    #budget
    b = budget_level

    def cost_param_initialize(model, i):
        return(c_i_dict.get(i))

    model.c_i = Param(model.I, initialize = cost_param_initialize)

    #model.c_k_i.pprint()

    def budget_param_initialize(model):
        return(b)

    model.b = Param(initialize = budget_param_initialize)

    ########inventory availability parameters##########
    #starting inventory
    z_i_init_dict = {
        1 : 5000,
        2 : 5000,
        3 : 5000,
        4 : 5000
    }

    def beg_inv_param_initialize(model, i):
        return(z_i_init_dict.get(i))

    model.z_i_init = Param(model.I, initialize = beg_inv_param_initialize)

    #######Supply parameters########
    #incoming orders
    o_i_t_init_dict = {}


    for i in range(1,I_set+1):
        for t in range(1,T_set+1):
            if t <= 2:
                o_i_t_init_dict[tuple([i,t])] = 0 
            else:
                o_i_t_init_dict[tuple([i,t])] = 0



    #lead times
    lead_time_dictionary_supplier_SKU = {1: [3,6,4], #min, max, likely, a,b,c
                         2: [2,5,3], 
                         3: [1,3,2],
                         4: [0,0,1]}

    def lead_time_dist(a,b,c):
        lead_time_dist_array = np.zeros(T_set+1)
        cum_prob = 0
        t = 0

        while t < b+1:
            if t+1 <= a:
                t = t+1 
            elif (a < t+1 & t+1 <= c):
                cum_prob_temp = ((t+1-a)**2)/((b+1-a)*(c-a))
                prob = cum_prob_temp - cum_prob
                lead_time_dist_array[t] = prob
                cum_prob = cum_prob_temp
                t = t+1
            elif c < t+1 & t+1 <= b+1:
                cum_prob_temp = 1-((b+1-(t+1))**2)/((b+1-a)*(b+1-c))
                prob = cum_prob_temp - cum_prob
                lead_time_dist_array[t] = prob
                cum_prob = cum_prob_temp
                t = t+1

        return(lead_time_dist_array)

    f_i_l_dict = {}
    for i in model.I:
        lead_time_dist_array = lead_time_dist(lead_time_dictionary_supplier_SKU.get(i)[0],
                                             lead_time_dictionary_supplier_SKU.get(i)[1],
                                             lead_time_dictionary_supplier_SKU.get(i)[2])
        for t in model.T_0:
            f_i_l_dict[tuple([i,t])] = lead_time_dist_array[t]


    #supply capacities
    cap_dictionary_supplier_SKU = {1: np.repeat([22000,24000], [1,T_set-1]), #min, max, likely, a,b,c
                         2: np.repeat([24000,30000], [1,T_set-1]), 
                         3: [24000]*T_set,
                         4: [30000]*T_set}

    cap_dictionary_supplier_SKU_t = {}

    for i in model.I:
        for t in model.T:
            cap_dictionary_supplier_SKU_t[tuple([i,t])] = cap_dictionary_supplier_SKU[i][t-1]


    #supplier-SKU names
    names_supplier_SKU = ['Supplier 1 - SKU: N95-MG', 'Supplier 1 - SKU: N95-N', 
     'Supplier 2 - SKU: SUR-MG', 'Supplier 2 - SKU: SUR-N']

    def incoming_orders_initialize(model, i, t):
        return(o_i_t_init_dict.get(tuple([i,t])))

    model.o_i_t_init = Param(model.I, model.T, initialize = incoming_orders_initialize)

    #model.o_k_i_t.pprint()

    def delay_dist_param_initialize(model, i, t):
        return(f_i_l_dict.get(tuple([i,t])))

    model.f_i_l = Param(model.I, model.T_0, 
                              initialize = delay_dist_param_initialize)

    def supplier_cap_param_initialize(model, i, t):
        return(cap_dictionary_supplier_SKU_t[i,t])

    model.cap_i_t = Param(model.I, model.T, initialize = supplier_cap_param_initialize)

    #model.cap_k_i_t.pprint()

    f_i_l_dict = {}
    i = 1
    #for i in model.I:
    lead_time_dist_array = lead_time_dist(lead_time_dictionary_supplier_SKU.get(i)[0],
                                         lead_time_dictionary_supplier_SKU.get(i)[1],
                                         lead_time_dictionary_supplier_SKU.get(i)[2])

    #print(lead_time_dist_array)
    #for t in model.T_0:
    #    f_i_l_dict[tuple([i,t])] = lead_time_dist_array[t]

    lead_time_dictionary_supplier_SKU.get(i)

    f_i_l_dict = {}
    i = 1
    #for i in model.I:
    lead_time_dist_array = lead_time_dist(lead_time_dictionary_supplier_SKU.get(i)[0],
                                         lead_time_dictionary_supplier_SKU.get(i)[1],
                                         lead_time_dictionary_supplier_SKU.get(i)[2])

    #model.T_0.pprint()

    #model.I.pprint()

    ######demand parameters#########

    #unfulfilled demand
    n_k_init_dict = {
        1 : 0
    }

    #demand reliability level
    epsilon = reliability_level

    #projected incoming demand
    base_demand_k = {1: [25000, 45000, 35000]}

    #used in this example to adjust demand RV over time
    dynamic_adj_dem_t = {1: [0.5, 0.55, 0.6, 1, 1.1, 1.5, 1.6, 1.2, 1.1, 0.85, .8, .8]}

    def triangular_dist_demand(epsilon, k, t):
        base_demand_ktemp = base_demand_k[k]
        adjust_demand_temp = dynamic_adj_dem_t[k][t-1]
        a = base_demand_ktemp[0]*adjust_demand_temp
        b = base_demand_ktemp[1]*adjust_demand_temp
        c = base_demand_ktemp[2]*adjust_demand_temp

        if epsilon <= (c-a)/(b-a):
            return((a+((((epsilon)*(b-a)*(c-a)))**(1/2))))
        else:
            return((b-(((1-epsilon)*(b-a)*(b-c))**(1/2))))


    def unfulfilled_demand_param_initialize(model, k):
        return(n_k_init_dict.get(k))

    model.n_k_init = Param(model.K, initialize = unfulfilled_demand_param_initialize)

    #model.n_k_intialize.pprint()

    #assume preparing for one sd above the mean
    def demand_param_initialize(model, k, t):
        temp = triangular_dist_demand(epsilon, k,t)
        return(temp)

    model.d_k_t = Param(model.K, model.T, initialize = demand_param_initialize)

    ####initialize variables####
    model.x_i_t = Var(model.I, model.T, within = NonNegativeReals) #amount ordered
    model.y_i_t = Var(model.I, model.T, within = NonNegativeReals) #amount recieved
    model.z_i_t = Var(model.I, model.T, within = NonNegativeReals)
    model.n_k_t = Var(model.K, model.T, within = NonNegativeReals)
    model.m_k_i_t = Var(model.K, model.I, model.T, within = NonNegativeReals)
    
    return(model)

In [5]:
def initialize_objective(model):
    model.Objective = Objective(expr = 
                            (sum(model.q_k_i[k,i]*model.tau_k_t[k,t]*model.m_k_i_t[k,i,t] 
                                                      for k in model.K for t in model.T for i in model.I_K[k])),
                            sense = maximize)
    
    return(model)

In [6]:
def initialize_constraints(model):
    def initalize_unsatisfied_demand_constraint_initialize(model, k):
        return(model.n_k_t[k,1] == model.n_k_init[k])

    model.initalize_unsatisfied_demand_constraint= \
    Constraint(model.K, rule = initalize_unsatisfied_demand_constraint_initialize)

    def calculate_unsatisfied_demand_constraint_initialize(model,k,t):
        if (t == 1):
            #return(None)
            return(sum(model.m_k_i_t[k,i,t] for i in model.I_K[k]) <= model.n_k_t[k,t] + model.d_k_t[k,t])
        else:
            #return(None)
            return(model.n_k_t[k,t] == model.n_k_t[k,t-1] + model.d_k_t[k,t] \
                   - sum(model.m_k_i_t[k,i,t] for i in model.I_K[k]))

    model.calculate_unsatisfied_demand_constraint = \
    Constraint(model.K, model.T, rule = calculate_unsatisfied_demand_constraint_initialize)

    def supplier_constraint_initialize(model, i, t):
        return(model.x_i_t[i,t] <= model.cap_i_t[i,t])

    model.supplier_constraint = \
    Constraint( model.I, model.T, rule = supplier_constraint_initialize)

    def incoming_orders_constraint_initialize(model, i, t):
        return(model.y_i_t[i,t] - 
               sum(model.f_i_l[i, t-t_ordered_time]*model.x_i_t[i,t_ordered_time] 
                   for t_ordered_time in range(1,t+1)) - model.o_i_t_init[i,t] == 0)

    model.incoming_ordered_constraint = Constraint(model.I, model.T, 
                                                   rule = incoming_orders_constraint_initialize)

    def beggining_inventory_constraint_initialize(model, i):
        return(model.z_i_t[i,1] == model.z_i_init[i])

    model.beggining_inventory_constraint = Constraint(model.I, 
                                                      rule = beggining_inventory_constraint_initialize)

    def calculate_available_inventory_constraint_initialize(model, i, t):
        if (t > 1):
            return(model.z_i_t[i,t] == model.z_i_t[i,t-1] + model.y_i_t[i,t-1] - \
                   sum(model.m_k_i_t[k,i,t-1] for k in model.K))
        else:
            return(Constraint.Skip)

    model.calculate_available_inventory_constraint = Constraint(model.I, model.T,
                                                                rule = 
                                                                calculate_available_inventory_constraint_initialize)

    def cannot_fulfill_more_than_available_constraint_initialize(model, i, t):
        return(sum(model.m_k_i_t[k,i,t] for k in model.K) - model.z_i_t[i,t] <= 0)

    model.cannot_fulfill_more_than_available_constraint = \
    Constraint(model.I, model.T, rule = cannot_fulfill_more_than_available_constraint_initialize)

    def warehouse_capacity_constraint_initialize(model, t):
        return(sum(model.s_i[i]*model.z_i_t[i,t] for i in model.I) <= model.h)

    model.warehouse_capacity_constraint = \
    Constraint(model.T, rule = warehouse_capacity_constraint_initialize)

    def budget_constraint_initialize(model):
        return(sum(model.c_i[i]*model.x_i_t[i,t] 
                   for i in model.I for t in model.T) 
               <= model.b)

    model.budget_constraint = \
    Constraint(rule = budget_constraint_initialize)
    
    return(model)

In [7]:
def extract_data(model, reliability_level, budget, 
                 unsatisfied_df, fulfilled_df, ordered_df, actual,
                HCC_dict):
    
    #extract from unsatisfied_df
    def unsatisfied_df_update(unsatisfied_df):
        if unsatisfied_df.empty:
            unsatisfied_df = pd.DataFrame(list(model.n_k_t.extract_values().items()),\
                                          columns = ['sets','units_unsatisfied'])
            unsatisfied_df = unsatisfied_df[unsatisfied_df['units_unsatisfied'].notnull()]            
            unsatisfied_df.loc[:,'item_type']=unsatisfied_df.sets.map(lambda x:x[0])
            unsatisfied_df.loc[:,'time_interval']=unsatisfied_df.sets.map(lambda x:x[1])
            unsatisfied_df = unsatisfied_df[['item_type', 'time_interval', 
                                             'units_unsatisfied']]
            unsatisfied_df ['reliability_level'] = [reliability_level]*len(unsatisfied_df)
            unsatisfied_df['budget'] = [budget]*len(unsatisfied_df)
            return(unsatisfied_df)
        else:
            unsatisfied_df_temp = pd.DataFrame(list(model.n_k_t.extract_values().items()),\
                                          columns = ['sets','units_unsatisfied'])
            unsatisfied_df_temp.loc[:,'item_type']=unsatisfied_df_temp.sets.map(lambda x:x[0])
            unsatisfied_df_temp.loc[:,'time_interval']=unsatisfied_df_temp.sets.map(lambda x:x[1])
            unsatisfied_df_temp = unsatisfied_df_temp[['item_type', 'time_interval', 
                                                       'units_unsatisfied']]
            unsatisfied_df_temp ['reliability_level'] = [reliability_level]*len(unsatisfied_df_temp)
            unsatisfied_df_temp['budget'] = [budget]*len(unsatisfied_df_temp)
            unsatisfied_df_temp = unsatisfied_df_temp[unsatisfied_df_temp['units_unsatisfied'].notnull()]

            return(unsatisfied_df.append(unsatisfied_df_temp, ignore_index=True))
        
    #extract from satisfied_df
    def fulfilled_df_update(fulfilled_df):
        if fulfilled_df.empty:
            fulfilled_df = pd.DataFrame(list(model.m_k_i_t.extract_values().items()),columns = ['sets','units_fulfilled'])
            fulfilled_df.loc[:,'item_type']=fulfilled_df.sets.map(lambda x:x[0])
            fulfilled_df.loc[:,'supplier']=fulfilled_df.sets.map(lambda x:x[1])
            fulfilled_df.loc[:,'time_interval']=fulfilled_df.sets.map(lambda x:x[2])
            fulfilled_df['reliability_level'] = [reliability_level]*len(fulfilled_df)
            fulfilled_df['budget'] = [budget]*len(fulfilled_df)
            fulfilled_df = fulfilled_df[fulfilled_df['units_fulfilled'].notnull()]
            return(fulfilled_df)
        else:
            fulfilled_df_temp = pd.DataFrame(list(model.m_k_i_t.extract_values().items()),columns = ['sets','units_fulfilled'])
            fulfilled_df_temp.loc[:,'item_type']=fulfilled_df_temp.sets.map(lambda x:x[0])
            fulfilled_df_temp.loc[:,'supplier']=fulfilled_df_temp.sets.map(lambda x:x[1])
            fulfilled_df_temp.loc[:,'time_interval']=fulfilled_df_temp.sets.map(lambda x:x[2])
            fulfilled_df_temp['reliability_level'] = [reliability_level]*len(fulfilled_df_temp)
            fulfilled_df_temp['budget'] = [budget]*len(fulfilled_df_temp)
            fulfilled_df_temp = fulfilled_df_temp[fulfilled_df_temp['units_fulfilled'].notnull()]

            return(fulfilled_df.append(fulfilled_df_temp, ignore_index=True))
        
    def ordered_df_update(ordered_df):
        if ordered_df.empty:
            ordered_df = pd.DataFrame(list(model.x_i_t.extract_values().items()),columns = ['sets','units_to_order'])
            ordered_df.loc[:,'supplier']=ordered_df.sets.map(lambda x:x[0])
            ordered_df.loc[:,'time_interval']=ordered_df.sets.map(lambda x:x[1])
            ordered_df['reliability_level'] = [reliability_level]*len(ordered_df)
            ordered_df['budget'] = [budget]*len(ordered_df)
            
            return(ordered_df)
        else:
            ordered_df_temp = pd.DataFrame(list(model.x_i_t.extract_values().items()),columns = ['sets','units_to_order'])
            ordered_df_temp.loc[:,'supplier']=ordered_df_temp.sets.map(lambda x:x[0])
            ordered_df_temp.loc[:,'time_interval']=ordered_df_temp.sets.map(lambda x:x[1])
            ordered_df_temp['reliability_level'] = [reliability_level]*len(ordered_df_temp)
            ordered_df_temp['budget'] = [budget]*len(ordered_df_temp)
            
            return(ordered_df.append(ordered_df_temp, ignore_index = True))
    
    HCC_dict[tuple([budget, reliability_level])] = model.Objective.expr()
    
    unsatisfied_df = unsatisfied_df_update(unsatisfied_df)
    fulfilled_df = fulfilled_df_update(fulfilled_df)
    ordered_df = ordered_df_update(ordered_df)
    
    return(unsatisfied_df, fulfilled_df, ordered_df, HCC_dict)

In [8]:
fulfilled_df_plan = pd.DataFrame()
unsatisfied_df_plan = pd.DataFrame()
ordered_df_plan = pd.DataFrame()
HCC_dict = {}

for r in reliability_levels:
    
    #objective_itr = [0,1]
    #b = 1
    #while(objective_itr[b-1] < objective_itr[b]): 
    for b in budget_levels:
        model = initialize_model(r, b)
        model = initialize_objective(model)
        model = initialize_constraints(model)
        #opt = SolverFactory('gurobi_persistent')
        #opt.set_instance(model)
        #opt.solve(model)
        opt = pyomo.opt.SolverFactory("glpk")
        results = opt.solve(model)
        opt.solve(model)
        
        #if (objective_itr[b-1] < objective_itr[b]):
            #extract data
        trade_off_dict[tuple([r,b])] = value(model.Objective) #model.Objective.value()
        #objective_itr.append(value(model.Objective))#model.Objective.value())

        unsatisfied_df_plan, fulfilled_df_plan, ordered_df_plan, HCC_dict = \
        extract_data(model, r, b, unsatisfied_df_plan, 
                     fulfilled_df_plan, ordered_df_plan, 'NA',
                     HCC_dict)
        
        #b = b + 1

In [9]:
#set up delay df to calculate average delay
#group filfilled and unsatisfied demand over item type (over all supplier-SKUs)
delay_ref_df = fulfilled_df_plan.groupby(['item_type', 'time_interval', 
                      'reliability_level', 'budget'])['units_fulfilled'].sum().reset_index()

#create a unique id to calculate cum fulfilled (for calculating delay) based on reliability level, budget and item type
delay_ref_df['unique_id'] = delay_ref_df.groupby(['reliability_level', 'budget', 'item_type']).ngroup()
delay_ref_df = delay_ref_df.sort_values(by=['unique_id', 'time_interval', 'item_type'])
delay_ref_df = delay_ref_df.reset_index(drop = True)

unique_id_itr = 0
cum_units_fulfilled_array = np.zeros(len(delay_ref_df))

for row in range(len(delay_ref_df)):
    units_fulfilled_temp = delay_ref_df['units_fulfilled'][row]
    
    if delay_ref_df['unique_id'][row] == unique_id_itr:
        cum_units_fulfilled_array[row] = units_fulfilled_temp+cum_units_fulfilled_array[row-1]
    else:
        cum_units_fulfilled_array[row] = units_fulfilled_temp 
        
    unique_id_itr = delay_ref_df['unique_id'][row]

def triangular_dist_demand(epsilon, k, t):
        base_demand_ktemp = base_demand_k[k]
        adjust_demand_temp = dynamic_adj_dem_t[k][t-1]
        a = base_demand_ktemp[0]*adjust_demand_temp
        b = base_demand_ktemp[1]*adjust_demand_temp
        c = base_demand_ktemp[2]*adjust_demand_temp

        if epsilon <= (c-a)/(b-a):
            return((a+((((epsilon)*(b-a)*(c-a)))**(1/2))))
        else:
            return((b-(((1-epsilon)*(b-a)*(b-c))**(1/2))))
        
demand_array = np.zeros((len(reliability_levels)*K_set*T_set))
reliability_array = np.zeros((len(reliability_levels)*K_set*T_set))
K_array = np.zeros((len(reliability_levels)*K_set*T_set))
T_array = np.zeros((len(reliability_levels)*K_set*T_set))
cum_demand_array = np.zeros((len(reliability_levels)*K_set*T_set))

#projected incoming demand
base_demand_k = {1: [25000, 45000, 35000]}

#used in this example to adjust demand RV over time
dynamic_adj_dem_t = {1: [0.5, 0.55, 0.6, 1, 1.1, 1.5, 1.6, 1.2, 1.1, 0.85, .8, .8]}

n = 0
#in this case iterate over reliability level 
#because that will be what changes in the for loop when looking at a new set of demand
reliability_current_itr = 0

for r in range(len(reliability_levels)):
    for k in range(K_set):
        for t in range(T_set):
            demand_array[n] = triangular_dist_demand(reliability_levels[r], k+1, t+1)
            if reliability_current_itr == reliability_levels[r]:
                cum_demand_array[n] = demand_array[n]+cum_demand_array[n-1]
            else:
                cum_demand_array[n] = demand_array[n]
            reliability_array[n] = reliability_levels[r]
            K_array[n] = k+1
            T_array[n] = t+1
            
            n = n+1
            reliability_current_itr = reliability_levels[r]
            
demand_df = pd.DataFrame()
demand_df['item_type'] = K_array
demand_df['time_interval'] = T_array
demand_df['reliability_level'] = reliability_array
demand_df['incoming_demand'] = demand_array
#demand_df['cum_demand'] = cum_demand_array

delay_ref_df = pd.merge(delay_ref_df, demand_df,
                    how = 'left', on = ['item_type','time_interval',
                                        'reliability_level'])

In [10]:
demand_fulfillment_schedule_df = pd.DataFrame(columns=['unique_id', 'time_interval', 
                                                      'units_fulfilled', 'incoming_demand'])

for newcol in range(1,T_set+1):
    demand_fulfillment_schedule_df[newcol] = ""
    
for unique_id_itr in delay_ref_df['unique_id'].unique():
    delay_ref_df_temp = delay_ref_df[delay_ref_df['unique_id'] == unique_id_itr]
    fulfilled_remaining = (delay_ref_df_temp['units_fulfilled']).to_numpy()
    incoming_demand_remaining = (delay_ref_df_temp['incoming_demand']).to_numpy()
    
    for t1 in range(1,T_set+1):
        demand_fulfillment_schedule_df = demand_fulfillment_schedule_df.append({'unique_id': unique_id_itr,
                                                                                'time_interval': t1,
                                                                                'units_fulfilled': fulfilled_remaining[t1-1],
                                                                                'incoming_demand': incoming_demand_remaining[t1-1]},
                                                                               #'remaining_demand': incoming_demand_remaining[t1-1],
                                                                               #'remaining_units_fulfilled': fulfilled_remaining[t1-1]},
                                                                               ignore_index=True)  
        
demand_fulfillment_schedule_df = demand_fulfillment_schedule_df.fillna(0)

In [11]:
for unique_id_itr in delay_ref_df['unique_id'].unique():
    delay_ref_df_temp = delay_ref_df[delay_ref_df['unique_id'] == unique_id_itr]
    fulfilled_orig_temp = (delay_ref_df_temp['units_fulfilled']).to_numpy()
    demand_orig_temp = (delay_ref_df_temp['incoming_demand']).to_numpy()
    
    time_requested = 1
    time_fulfilled = 1
    
    while time_requested <= T_set and time_fulfilled <= T_set:
        
        #calculate the amount of the fulfilled demand accounted for at time_fulfilled
        demand_fulfillment_schedule_df_temp = demand_fulfillment_schedule_df[demand_fulfillment_schedule_df['unique_id']==unique_id_itr]
        amt_fulfilled_accounted_for = sum(demand_fulfillment_schedule_df_temp[time_fulfilled])
        
        fulfilled_remaining = fulfilled_orig_temp[time_fulfilled-1]-amt_fulfilled_accounted_for
        
        #calculate the amount of demand accounted for at time_requested
        demand_index = demand_fulfillment_schedule_df[(demand_fulfillment_schedule_df['time_interval'] == time_requested) &
                                      (demand_fulfillment_schedule_df['unique_id'] == unique_id_itr)].index[0]
        amt_demand_fulfilled = sum(demand_fulfillment_schedule_df.iloc[demand_index,-T_set:])
        
        demand_remaining = demand_orig_temp[time_requested-1]-amt_demand_fulfilled
        
        
        #the amount fulfilled at during time interval time_fulfilled, 
        #that was orig. requested at time time_requested is the min of fulfilled and demand remaining
        
        amt_fulfilled = min(fulfilled_remaining, demand_remaining)
        
        demand_fulfillment_schedule_df.loc[(demand_fulfillment_schedule_df['time_interval'] == time_requested) \
                                   & (demand_fulfillment_schedule_df['unique_id'] == unique_id_itr),\
                                   time_fulfilled] = amt_fulfilled
        
            
            
        #if there is no more remaining demand requests unassigned orig. requested at time time_requested
        #AND there is no more unassigned fulfilled demand from time time_fulfilled
        if (fulfilled_remaining == demand_remaining):
            time_requested = time_requested+1
            time_fulfilled = time_fulfilled+1
            
        #if there is remaining demand requests, but no more fulfilled demand
        if (fulfilled_remaining < demand_remaining):
            time_fulfilled = time_fulfilled+1
            
        #if there is remaining fulfilled demand, but no more demand requests
            
        if (fulfilled_remaining > demand_remaining):
            time_requested = time_requested+1
        

demand_fulfillment_schedule_df['units_UNfulfilled'] = demand_fulfillment_schedule_df['incoming_demand']-\
demand_fulfillment_schedule_df.loc[:,demand_fulfillment_schedule_df.columns[-T_set:]].sum(1)

In [12]:
demand_fulfillment_schedule_df.to_csv('demand_fulfillment_schedule_df.csv', index=False)

In [13]:
os.getcwd()

'/Users/chelseagreene/github/epi_supplychain_optimization/inventory_management/model/publication/journal_of_humanitarian_log_and_scm/outputs/strategic_decisions'

In [14]:
demand_fulfillment_schedule_df[13] = demand_fulfillment_schedule_df['units_UNfulfilled']

In [15]:
#calculate average delay from delay fulfillment schedule
delay_calcs_df=pd.melt(demand_fulfillment_schedule_df, 
                       id_vars=['unique_id', 'time_interval',
                                'units_fulfilled', 'incoming_demand', 'units_UNfulfilled'])
delay_calcs_df = delay_calcs_df.rename(columns={"variable": "time_interval_fulfilled", 
                               "time_interval": "time_interval_requested"})

delay_calcs_df=delay_calcs_df[['unique_id', 'time_interval_fulfilled', 'time_interval_requested', 'value']]
delay_calcs_df = delay_calcs_df[delay_calcs_df['value']>0]
delay_calcs_df['delay'] = delay_calcs_df['time_interval_fulfilled']-delay_calcs_df['time_interval_requested']
delay_calcs_df.loc[delay_calcs_df['delay']>0, 'is_delayed'] = 1
delay_calcs_df.loc[delay_calcs_df['delay']==0, 'is_delayed'] = 0
delay_calcs_df.loc[delay_calcs_df['delay']>1, 'is_delayed_2'] = 1
delay_calcs_df.loc[delay_calcs_df['delay']<=1, 'is_delayed_2'] = 0
delay_calcs_df.loc[delay_calcs_df['delay']>2, 'is_delayed_3'] = 1
delay_calcs_df.loc[delay_calcs_df['delay']<=2, 'is_delayed_3'] = 0

In [16]:
delay_calcs_df1 = delay_calcs_df.groupby(by = ['unique_id', 'is_delayed'], as_index=False).agg({"value": "sum"})
delay_calcs_df1['amount_delayed'] = delay_calcs_df1['value']*delay_calcs_df1['is_delayed']
delay_calcs_df1 = delay_calcs_df1.groupby(by = ['unique_id'], as_index=False).agg({"value": "sum",
                                                                                  "amount_delayed": "sum"})
delay_calcs_df1['percent_delayed'] = delay_calcs_df1['amount_delayed']/delay_calcs_df1['value']
delay_calcs_df1 = delay_calcs_df1[['unique_id', 'percent_delayed']]


delay_calcs_df2 = delay_calcs_df.groupby(by = ['unique_id', 'is_delayed_2'], as_index=False).agg({"value": "sum"})
delay_calcs_df2['amount_delayed'] = delay_calcs_df2['value']*delay_calcs_df2['is_delayed_2']
delay_calcs_df2 = delay_calcs_df2.groupby(by = ['unique_id'], as_index=False).agg({"value": "sum",
                                                                                  "amount_delayed": "sum"})
delay_calcs_df2['percent_delayed_by_at_least_2'] = delay_calcs_df2['amount_delayed']/delay_calcs_df2['value']
delay_calcs_df2 = delay_calcs_df2[['unique_id', 'percent_delayed_by_at_least_2']]


delay_calcs_df3 = delay_calcs_df.groupby(by = ['unique_id', 'is_delayed_3'], as_index=False).agg({"value": "sum"})
delay_calcs_df3['amount_delayed'] = delay_calcs_df3['value']*delay_calcs_df3['is_delayed_3']
delay_calcs_df3 = delay_calcs_df3.groupby(by = ['unique_id'], as_index=False).agg({"value": "sum",
                                                                                  "amount_delayed": "sum"})
delay_calcs_df3['percent_delayed_by_at_least_3'] = delay_calcs_df3['amount_delayed']/delay_calcs_df3['value']
delay_calcs_df3 = delay_calcs_df3[['unique_id', 'percent_delayed_by_at_least_3']]

In [17]:
delay_calcs_df_metric = delay_calcs_df1.merge(delay_calcs_df2, on='unique_id', how='left')
delay_calcs_df_metric = delay_calcs_df_metric.merge(delay_calcs_df3, on='unique_id', how='left')

In [18]:
delay_calcs_df_metric

,unique_id,percent_delayed,percent_delayed_by_at_least_2,percent_delayed_by_at_least_3
0,0.0,0.595556,0.468627,0.395567
1,1.0,0.431102,0.287971,0.214888
2,2.0,0.222918,0.098089,0.029078
3,3.0,0.043571,0.000000,0.000000
4,4.0,0.055419,0.000000,0.000000
5,5.0,0.601936,0.494187,0.412558
6,6.0,0.477945,0.317242,0.244893
7,7.0,0.343028,0.146486,0.071115
8,8.0,0.131704,0.000000,0.000000
9,9.0,0.108329,0.000000,0.000000


In [19]:
#average quality of fulfilled items

q_k_i_dict = {
    tuple([1,1]):1,
    tuple([1,2]):.9,
    tuple([1,3]):.8,
    tuple([1,4]):.7
}

quality_array = np.empty(len(fulfilled_df_plan))

for i in fulfilled_df_plan.index:
    it = fulfilled_df_plan['item_type'][i]
    ss = fulfilled_df_plan['supplier'][i]
    quality_array[i] = q_k_i_dict.get(tuple([it, ss]))
    
fulfilled_df_plan['quality'] = quality_array

fulfilled_df_quality = \
fulfilled_df_plan[fulfilled_df_plan['units_fulfilled']>0]

fulfilled_df_quality['weighted_quality'] = fulfilled_df_quality['quality']*\
fulfilled_df_quality['units_fulfilled']

fulfilled_df_quality_metric = fulfilled_df_quality\
.groupby(by = ['reliability_level', 'budget'], as_index = False).\
agg({"weighted_quality": "sum",
    "units_fulfilled": "sum"})

fulfilled_df_quality_metric['average_quality'] = \
fulfilled_df_quality_metric['weighted_quality']/\
fulfilled_df_quality_metric['units_fulfilled']

<ipython-input-19-be991c509157>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fulfilled_df_quality['weighted_quality'] = fulfilled_df_quality['quality']*\


In [20]:
fulfilled_df_quality_metric

,reliability_level,budget,weighted_quality,units_fulfilled,average_quality
0,0.50,1000000,159210.000000,189400.000000,0.840602
1,0.50,1500000,227307.777778,262755.555556,0.865092
2,0.50,2000000,298338.888889,338194.444444,0.882152
3,0.50,2500000,361601.366120,406000.000000,0.890644
4,0.50,3000000,365535.850694,406000.000000,0.900335
5,0.75,1000000,164921.417767,198186.796564,0.832151
6,0.75,1500000,233019.195545,271542.352120,0.858132
7,0.75,2000000,304776.404619,348000.325870,0.875793
8,0.75,2500000,378458.670202,426218.928900,0.887944
9,0.75,3000000,390369.916207,439975.613382,0.887254


In [21]:
delay_ref_df

,item_type,time_interval,reliability_level,budget,units_fulfilled,unique_id,incoming_demand
0,1,1,0.50,1000000,17500.000000,0,17500.000000
1,1,2,0.50,1000000,19250.000000,0,19250.000000
2,1,3,0.50,1000000,21000.000000,0,21000.000000
3,1,4,0.50,1000000,34500.000000,0,35000.000000
4,1,5,0.50,1000000,39000.000000,0,38500.000000
...,...,...,...,...,...,...,...
175,1,8,0.95,3000000,50000.000000,14,50205.266808
176,1,9,0.95,3000000,50000.000000,14,46021.494574
177,1,10,0.95,3000000,50000.000000,14,35562.063989
178,1,11,0.95,3000000,46265.761556,14,33470.177872


In [22]:
quality_delay_metric = delay_ref_df[['unique_id', 'reliability_level', 'budget']].\
drop_duplicates(subset=['unique_id'])

quality_delay_metric = quality_delay_metric[['unique_id', 'reliability_level', 'budget']]\
.merge(fulfilled_df_quality_metric[['reliability_level', 'budget', 'average_quality']], 
       on=['reliability_level', 'budget'], how='left')

quality_delay_metric = quality_delay_metric.merge(delay_calcs_df_metric, on = ['unique_id'], how = 'left')

In [23]:
quality_delay_metric

,unique_id,reliability_level,budget,average_quality,percent_delayed,percent_delayed_by_at_least_2,percent_delayed_by_at_least_3
0,0,0.50,1000000,0.840602,0.595556,0.468627,0.395567
1,1,0.50,1500000,0.865092,0.431102,0.287971,0.214888
2,2,0.50,2000000,0.882152,0.222918,0.098089,0.029078
3,3,0.50,2500000,0.890644,0.043571,0.000000,0.000000
4,4,0.50,3000000,0.900335,0.055419,0.000000,0.000000
5,5,0.75,1000000,0.832151,0.601936,0.494187,0.412558
6,6,0.75,1500000,0.858132,0.477945,0.317242,0.244893
7,7,0.75,2000000,0.875793,0.343028,0.146486,0.071115
8,8,0.75,2500000,0.887944,0.131704,0.000000,0.000000
9,9,0.75,3000000,0.887254,0.108329,0.000000,0.000000


In [ ]:
fulfilled_df_item_time = fulfilled_df_plan.groupby(['item_type', 'time_interval', 
                      'reliability_level', 'budget'])['units_fulfilled'].sum().reset_index()

unsatisfied_df_item_time = unsatisfied_df_plan.groupby(['item_type', 'time_interval', 
                      'reliability_level', 'budget'])['units_unsatisfied'].sum().reset_index()

reliability_df = pd.merge(fulfilled_df_item_time[['item_type', 'time_interval', 'units_fulfilled',
                                                  'reliability_level', 'budget']], 
                          unsatisfied_df_item_time[['item_type', 'time_interval', 'units_unsatisfied',
                                          'reliability_level', 'budget']], 
                          how='left', on=['item_type','time_interval',
                                         'reliability_level', 'budget'])

reliability_df['total_number_of_open_requests'] = reliability_df['units_fulfilled'] +\
reliability_df['units_unsatisfied'] 

reliability_df['percent_unsatisfied'] = \
reliability_df['units_unsatisfied']/reliability_df['total_number_of_open_requests']

reliability_df_grouped = \
reliability_df.groupby(['reliability_level', 'budget'])['percent_unsatisfied'].sum().reset_index()

reliability_df = pd.merge(fulfilled_df_item_time[['item_type', 'time_interval', 'units_fulfilled',
                                                  'reliability_level', 'budget']], 
                          unsatisfied_df_plan[['item_type', 'time_interval', 'units_unsatisfied',
                                          'reliability_level', 'budget']], 
                          how='left', on=['item_type','time_interval',
                                         'reliability_level', 'budget'])

reliability_df_grouped['risk_of_understock'] = (reliability_df_grouped['percent_unsatisfied']/(T_set*K_set))

reliability_df_grouped['budget_thousands'] = reliability_df_grouped['budget']/1000
reliability_df_grouped = reliability_df_grouped[reliability_df_grouped['budget_thousands'] <= max(unsatisfied_df_plan['budget'])]
reliability_df_grouped['percent_unsatisfied'] = (reliability_df_grouped['risk_of_understock']*100).astype(int)


#set lengend title size of plots
plot.rcParams['legend.title_fontsize'] = 'Large'

In [ ]:
reliability_df_grouped

In [ ]:
fig, ax = plot.subplots(figsize=(6,6))


ax.plot('budget_thousands', 'percent_unsatisfied', 
          data=reliability_df_grouped[reliability_df_grouped['reliability_level'] == reliability_levels[0]],
         marker='X', color = 'black', label = '50%', markersize=8)#r'$\theta^D = \theta^{CAP}$ = 0' + ' (Assuming Expected Demand and Supplier Capacity)')
ax.plot('budget_thousands', 'percent_unsatisfied', 
          data=reliability_df_grouped[reliability_df_grouped['reliability_level'] == reliability_levels[1]],
         marker='o', color = 'black', label = '75%', markersize=8, mfc = 'white') #r'$\theta^D = \theta^{CAP}$ = 1' + ' (Assuming Some Variability in Demand and Supplier Capacity)')
ax.plot('budget_thousands', 'percent_unsatisfied', 
          data=reliability_df_grouped[reliability_df_grouped['reliability_level'] == reliability_levels[2]],
         marker='D', color = 'gray', label = '95%', markersize = 8)#r'$\theta^D = \theta^{CAP}$ = 2' + ' (high reliability)')

ax.set_xlabel('Budget (in thousands)', fontsize = 13)
ax.set_ylabel('Overall Percent of Unsatisfied Demand', fontsize = 13)

#ax.set(ylim=(0, 1), xlim=(0, max(reliability_df_grouped['budget'])))
ax.set(ylim=(0, 100), xlim=(0, 2500))
#ax.set_title(r'$\bf{Overall ~ Expected ~Percent ~of ~Unsatisfied ~Demand}$'+
#             '\n'+ r'$\bf{Under ~Various~ Budgets ~and ~Variability~ in~Demand ~and ~Supplier~Capacity~Assumptions~}$' +
#             '\n \n \n where the expected percent of unsatisfied demand for each item during week'#+ r'$(t)$' + ', represented by: \n'+\
              #r'$\psi_{k,t} = \frac{\alpha_{k,t}}{\alpha_{k,t} + \sum_{i \in I} \beta_{k,i,t}} \forall k \in K, t \in T$' +
#             '\n ...is averaged over all items and time intervals to obtain the '#+  r'$(K)$'+ ' and time intervals '+  r'$(T)$'+ ' to obtain the:'\
#             '\n overall expected percent of unsatisfied demand',# + r'$\frac{\sum_{t \in T} \sum_{k \in K} \psi_{k,t}}{K \times T}$',
#             fontsize = 16)

ax.yaxis.set_major_formatter(mtick.PercentFormatter())

plot.legend(title = 'Demand and Supply \n Capacity Reliability', 
            fontsize=13, 
            fancybox=True)._legend_box.align='center'

plot.grid(color='gray', linestyle='-', linewidth=.4, axis = 'y')


#os.chdir(os.getcwd()+'/outputs/3_2_1_strategic_decisions')
#plot.savefig('budget_high_WH.jpg')
plot.savefig('budget_low_WH.jpg')